<a href="https://colab.research.google.com/github/unkovic1/AndroidApp-Kotlin-WishListApp/blob/master/Predikcija_sentimenata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Učitavanje fajla sa zaglavljem
file_path = "/content/IMDb_Movie_Reviews.csv"






In [ ]:
# Učitavanje CSV fajla sa ručno dodeljenim kolonama
df = pd.read_csv(file_path, header=None, names=['review', 'sentiment'])

In [ ]:
# Prikazivanje prvih nekoliko redova i imena kolona
print(df.head())
print(df.columns)
import nltk
from nltk.corpus import stopwords
import re

                                              review  sentiment
0                                                  0          1
1  I did not enjoy the film Eraser whatsoever. It...          0
2  Be very afraid of anyone who likes this film. ...          0
3  The 3rd and last big screen spin off from the ...          0
4  Barely three and a half years after just scrap...          1
Index(['review', 'sentiment'], dtype='object')


In [ ]:
# Preuzimanje stop-words (ako još nisi)
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Lista stop-words na engleskom
stop_words = set(stopwords.words('english'))

In [ ]:
# Funkcija za čišćenje teksta
def clean_text(text):
    # Pretvaranje u mala slova
    text = text.lower()

    # Uklanjanje specijalnih karaktera, brojeva i interpunkcija (koristimo regularne izraze)
    text = re.sub(r'[^a-z\s]', '', text)

    # Razbijanje teksta na reči i uklanjanje stop-reči
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Vraćanje čistog teksta kao string
    return ' '.join(words)


In [ ]:
# Primena funkcije na sve recenzije
df['cleaned_text'] = df['review'].apply(clean_text)  # 'review' je ime kolone sa recenzijama

In [ ]:
# Ispisivanje nekoliko čistih recenzija
print(df['cleaned_text'].head())
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

0                                                     
1    enjoy film eraser whatsoever awful acting bori...
2    afraid anyone likes film probably inhaled many...
3    rd last big screen spin popular itv sitcom ear...
4    barely three half years scraping months run oc...
Name: cleaned_text, dtype: object


In [ ]:
# Inicijalizacija TF-IDF vektorizatora
vectorizer = TfidfVectorizer(max_features=5000)  # Koristi samo 5000 najvažnijih reči

In [ ]:
# Primena vektorizacije na obučavajući skup
X = df['cleaned_text']  # Ulazni podaci (čiste recenzije)
y = df['sentiment']     # Izlazne vrednosti (sentiment - 0 ili 1)

In [ ]:
# Podela podataka na obučavajući i testirajući skup (70% obučavanje, 30% testiranje)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:

# Vektorizacija (pretvaranje recenzija u numerički format)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
# Ispisivanje oblika matrica (dimenzije)
print("Shape of X_train_tfidf:", X_train_tfidf.shape)
print("Shape of X_test_tfidf:", X_test_tfidf.shape)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

Shape of X_train_tfidf: (17500, 5000)
Shape of X_test_tfidf: (7501, 5000)


In [ ]:
# Kreiranje i treniranje modela
model = LogisticRegression(max_iter=1000)  # Povećaj max_iter ako model ne konvergira
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [ ]:

# Predviđanje na test skupu
y_pred = model.predict(X_test_tfidf)

In [ ]:
# Evaluacija modela
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8880149313424877
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.87      0.89      3761
           1       0.88      0.90      0.89      3740

    accuracy                           0.89      7501
   macro avg       0.89      0.89      0.89      7501
weighted avg       0.89      0.89      0.89      7501



In [ ]:
import joblib

# Sačuvaj TF-IDF vektorizator
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

# Sačuvaj model (logistic regression)
joblib.dump(model, 'sentiment_model.pkl')

print("Model i TF-IDF vektorizator su sačuvani!")

Model i TF-IDF vektorizator su sačuvani!


In [ ]:
import joblib

# Učitaj TF-IDF vektorizator i model
vectorizer = joblib.load('tfidf_vectorizer.pkl')
model = joblib.load('sentiment_model.pkl')


In [ ]:
# Funkcija za čišćenje novog teksta pre predikcije
def clean_new_text(text):
    # Pretvaranje u mala slova
    text = text.lower()

    # Uklanjanje specijalnih karaktera, brojeva i interpunkcija
    text = re.sub(r'[^a-z\s]', '', text)

    # Razbijanje teksta na reči i uklanjanje stop-reči
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Vraćanje čistog teksta kao string
    return ' '.join(words)

In [ ]:
# Funkcija za predviđanje sentimenta
def predict_sentiment(new_review):
    # Čišćenje novog komentara
    cleaned_review = clean_new_text(new_review)

    # Vektorizacija nove recenzije pomoću iste TF-IDF transformacije
    review_tfidf = vectorizer.transform([cleaned_review])

    # Predviđanje sentimenta (0 ili 1) pomoću modela
    sentiment = model.predict(review_tfidf)

    # Vraćanje rezultata kao "Pozitivna recenzija" ili "Negativna recenzija"
    return "Pozitivna recenzija" if sentiment[0] == 1 else "Negativna recenzija"

In [ ]:
# Testiranje sa novim komentarima
new_review = "  "
predicted_sentiment = predict_sentiment(new_review)
print(predicted_sentiment)  # Očekivani rezultat: "Pozitivna recenzija"

Negativna recenzija
